In [2]:
import numpy as np
import torch 
import torch.linalg as LA

from lib import *

In [3]:
%load_ext autoreload
%autoreload 2

### Проверка корректности функций

In [4]:
d = 3
inp_modes = [2, 3, 4]
out_modes = [2, 2, 2]
ranks = [1, 2, 2, 1]

t = TT_linear(inp_modes, out_modes, ranks)
W_cores = TensorTrain(t.W_cores, inp_modes, out_modes, ranks)
W = W_cores.to_tensor()

In [5]:
W_cores

shape: torch.Size([1, 2, 2, 2])
shape: torch.Size([2, 3, 2, 2])
shape: torch.Size([2, 4, 2, 1])

In [6]:
#Транспонирование

torch.allclose(W_cores.t().to_tensor(), W.t())

True

In [7]:
# Умножение на тензор

x = torch.rand(5, 24)
torch.allclose(x @ W, x * W_cores)

True

In [8]:
# Перемножение TT

torch.allclose((W_cores * W_cores.t()).to_tensor(), W @ W.t())

True

In [9]:
# Сложение TT

torch.allclose((W_cores + W_cores).to_tensor(), 2 * W)

True

In [10]:
# Норма

W_cores.norm()

4.6357661485672

In [11]:
# TT_SVD

torch.allclose(tt_svd_2D(W, inp_modes, out_modes, 10).to_tensor(), W, atol=1e-5)

True

In [12]:
# ортогонализация справа

W_cores_my_orth = W_cores._orthogonalize_right()
res = []
for core in W_cores_my_orth.cores[1:]:
    H = tt_methods._H(core)
    res.append(torch.allclose(H @ H.t(), torch.eye(H.shape[0]), atol=1e-6))
print(all(res))

True


In [13]:
# ортогонализация справа

W_cores_my_orth = W_cores._orthogonalize_left()
res = []
for core in W_cores_my_orth.cores[:-1]:
    V = tt_methods._V(core)
    res.append(torch.allclose(V.t() @ V, torch.eye(V.shape[1]), atol=1e-6))
print(all(res))

True


In [18]:
def _round(A, return_tt=True):
    cores = _orthogonalize_right(A, False)
    ranks = [1]
    for k in range(A.d - 1):
        U, S, V = LA.svd(_V(cores[k]), full_matrices=False)
        ranks.append(U.shape[1] - 1)
        U = U[:, 0:ranks[k + 1]]
        S = S[0:ranks[k + 1]]
        V = V[0:ranks[k + 1]]
        cores[k] = U.reshape(list(cores[k].shape[:-1]) + [ranks[k + 1]])
        cores[k+1] = (torch.diag(S) @ V @ _H(cores[k+1])).reshape([ranks[k+1]] + list(cores[k+1].shape[1:]))
    return TensorTrain(cores, A.inp_modes, A.out_modes, ranks) if return_tt else cores

In [263]:
# округление

Y_cores_round = _round(Y_cores)
Y = Y_cores_round.to_tensor().reshape(X.shape)
torch.allclose(Y, X, atol=1e-5)

False

In [267]:
Y_cores.norm(), Y_cores_round.norm()

(17.20688796043396, 12.020608961582184)